In [1]:
# Libraries for feature extraction:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from tqdm import tqdm

# Libraries for pre-processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [10]:
audio_dir = './Data/genres/'

In [11]:
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
          'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [37]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))

gen=list(genres.keys())

for i in tqdm(range(len(gen))):
    g=gen[i]
    pathlib.Path(f'./Spectrograms/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'{audio_dir}/{g}'):
        songname = f'{audio_dir}/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        plt.specgram(y, NFFT=1024, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'./Spectrograms/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:40<06:01, 40.22s/it]

 20%|██        | 2/10 [01:22<05:26, 40.87s/it]

 30%|███       | 3/10 [02:01<04:42, 40.36s/it]

 40%|████      | 4/10 [02:39<03:57, 39.52s/it]/home/maximiliano/.local/lib/python3.6/site-packages/matplotlib/axes/_axes.py:7739: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


 50%|█████     | 5/10 [03:18<03:17, 39.43s/it]

 60%|██████    | 6/10 [03:56<02:36, 39.02s/it]

 70%|███████   | 7/10 [04:36<01:57, 39.25s/it]

 80%|████████  | 8/10 [05:13<01:17, 38.54s/it]

 90%|█████████ | 9/10 [05:50<00:38, 38.21s/it]

100%|██████████| 10/10 [06:28<00:00, 38.19s/it]

<Figure size 720x720 with 0 Axes>

In [12]:
def get_features(y, sr, n_fft = 1024, hop_length = 512):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None, 'zcr': None, 'chroma':None}
    
    # Using librosa to calculate the features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rmse'] = librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    
    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
        return result
    
    dict_agg_features = get_moments(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [19]:
def read_process_songs(src_dir, debug = True):    
    # Empty array of dicts with the processed features from all files
    arr_features = []

    # Read files from the foldersfor root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name)
    for x,_ in genres.items():
        folder = src_dir + x
        
        
                
                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))
                
                # Append the result to the data structure
                features = get_features(signal, sr)
                features['genre'] = genres[x]
                arr_features.append(features)
    return arr_features

In [38]:
# Get list of dicts with features and convert to dataframe
features = read_process_songs(audio_dir,debug=True)

In [21]:
df_features = pd.DataFrame(features)

In [31]:
df_features.to_csv('./Features/features.csv', index=False)

In [32]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
centroid_mean    1000 non-null float64
centroid_std     1000 non-null float64
chroma_mean      1000 non-null float64
chroma_std       1000 non-null float64
flux_mean        1000 non-null float64
flux_std         1000 non-null float64
genre            1000 non-null int64
mfcc_0_mean      1000 non-null float64
mfcc_0_std       1000 non-null float64
mfcc_10_mean     1000 non-null float64
mfcc_10_std      1000 non-null float64
mfcc_11_mean     1000 non-null float64
mfcc_11_std      1000 non-null float64
mfcc_12_mean     1000 non-null float64
mfcc_12_std      1000 non-null float64
mfcc_1_mean      1000 non-null float64
mfcc_1_std       1000 non-null float64
mfcc_2_mean      1000 non-null float64
mfcc_2_std       1000 non-null float64
mfcc_3_mean      1000 non-null float64
mfcc_3_std       1000 non-null float64
mfcc_4_mean      1000 non-null float64
mfcc_4_std       1000 non-null float64

In [39]:
df_features.describe()

,centroid_mean,centroid_std,chroma_mean,chroma_std,flux_mean,flux_std,genre,mfcc_0_mean,mfcc_0_std,mfcc_10_mean,...,mfcc_8_std,mfcc_9_mean,mfcc_9_std,rmse_mean,rmse_std,roloff_mean,roloff_std,tempo,zcr_mean,zcr_std
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,2099.884503,651.228254,0.453887,0.290454,1.416929,1.554380,4.500000,-216.715982,59.474524,-5.556559,...,10.044678,7.913355,9.706690,0.128776,0.053664,4357.599098,1325.492437,121.097859,0.103601,0.054587
std,697.925111,279.687395,0.087931,0.018798,0.340395,0.677287,2.873719,98.037566,21.042905,6.773753,...,2.002480,7.863547,1.919358,0.064564,0.029613,1553.603633,522.318652,20.907121,0.041816,0.026082
min,534.046982,73.853662,0.211247,0.209446,0.608654,0.163264,0.000000,-617.812439,13.544660,-26.293921,...,6.103590,-12.293718,5.337916,0.005265,0.002197,730.280348,124.919782,69.837416,0.021691,0.007039
25%,1520.864167,440.409059,0.392546,0.280903,1.161329,1.077392,2.000000,-272.163597,44.713447,-10.481475,...,8.496050,1.855776,8.304847,0.084993,0.034438,3130.428957,918.601164,103.359375,0.070226,0.035771
50%,2111.869947,600.494099,0.460690,0.292580,1.363183,1.448499,4.500000,-192.257874,57.952559,-5.530820,...,9.807375,7.649071,9.436366,0.119896,0.048122,4477.282517,1272.714350,117.453835,0.099473,0.049468
75%,2613.219953,813.856939,0.510953,0.302484,1.632089,1.982040,7.000000,-147.033390,72.418177,-0.610548,...,11.298329,14.249839,10.839158,0.172889,0.065798,5345.809193,1670.144645,135.999178,0.131909,0.066533
max,4171.532799,1921.069855,0.689486,0.342938,2.450388,4.371769,9.000000,-47.043102,166.846832,17.014378,...,18.426645,27.413534,19.880682,0.394304,0.176083,8569.415467,3423.674142,184.570312,0.274505,0.186829
